# LangChain - pregunta a Marco Aurelio

In [1]:
from langchain.docstore.document import Document
import langchain.text_splitter as tsplitter

In [ ]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

## Importar datos

Leemos primero el documento, en este caso Meditaciones de Marco Aurelio.

In [2]:
with open("marcoaurelio.txt", "r") as f:
  data = f.read()

Tenemos que dividir el documento en trozos para poder convertirlos en embeddings

In [3]:
CHUNK_SIZE = 1000
text_splitter = tsplitter.RecursiveCharacterTextSplitter(chunk_size= CHUNK_SIZE, chunk_overlap = 0, length_function= len)


In [5]:
texts = text_splitter.create_documents([data])


In [11]:
print(f"We have {len(texts)} chunks")
print(texts[0].page_content)

We have 244 chunks
1. De mi abuelo Vero: el buen carácter y la serenidad.
2. De la reputación y memoria legadas por mi progenitor: el carácter
discreto y viril.
3. De mi madre: el respeto a los dioses, la generosidad y la abstención no
sólo de obrar mal, sino incluso de incurrir en semejante pensamiento; más
todavía, la frugalidad en el régimen de vida y el alejamiento del modo de
vivir propio de los ricos.
4. De mi bisabuelo: el no haber frecuentado las escuelas públicas y
haberme servido de buenos maestros en casa, y el haber comprendido que,
para tales fines, es preciso gastar con largueza.
5. De mi preceptor: el no haber sido de la facción de los Verdes ni de los
Azules, ni partidario de los parinularios ni de los escutarios; el soportar las
fatigas y tener pocas necesidades; el trabajo con esfuerzo personal y la
abstención de excesivas tareas, y la desfavorable acogida a la calumnia.
6. De Diogneto: el evitar inútiles ocupaciones; y la desconfianza en lo que


## Embeddings y base de datos

Inicializamos embeddings de OpenAI y Pinecone para BBDD vectorial

In [18]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [21]:
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)


In [22]:
index_name = "langchain1"

In [24]:
textssel = texts[:10]

10


Preparamos embeddings y los subimos a la BBDD

In [27]:
docsearch = Pinecone.from_texts([t.page_content for t in textssel], embeddings, index_name=index_name)

Probamos busqueda por similaridad, hemos usado métrica coseno (cosine similarity)

In [28]:
query = "Que concluyó de su padre?"
docs = docsearch.similarity_search(query, k=5)

## Parafraseo con OpenAI's gpt

In [30]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [32]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [34]:
query = "Que concluyó de su padre?"
docs = docsearch.similarity_search(query, k=5)
chain.run(input_documents = docs, question=query)

' Su padre fue descrito como teniendo mansedumbre y firmeza serena en las decisiones profundamente examinadas, no vanagloriarse con los honores aparentes, amor al trabajo y perseverancia, estar dispuesto a escuchar a los que podían hacer una contribución útil a la comunidad, distribuir sin vacilaciones a cada uno según su mérito, experiencia para distinguir cuando es necesario un esfuerzo sin desmayo y cuándo hay que relajarse, saber poner fin a las relaciones amorosas con los adolescentes, sociabilidad y consentir a los amigos que no asistieran siempre a sus comidas y que no le acompañaran necesariamente en sus desplazamientos, examen minucioso en las deliberaciones y tenacidad, sin eludir la indagación, satisfecho con las primeras impresiones, y celo por conservar los am'

## Alternativa en memoria (no pinecone)

In [50]:
model_path = "es_core_news_lg"
nlp = spacy.load(model_path)

# Define your document as a string in Spanish
document = texts[0].page_content
# Load the pre-trained Spanish word embeddings
nlp = spacy.load("es_core_news_lg")

def find_most_similar_chunk(text_chunks, question, k=5):
    # Parse the question with spaCy
    question_doc = nlp(question)

    # Create a dictionary to store the similarity scores
    similarity_scores = {}

    # Loop through each text chunk
    for chunk in text_chunks:
        # Parse the text chunk with spaCy
        chunk_doc = nlp(chunk)
        # Compute the similarity score between the question and the chunk
        similarity = question_doc.similarity(chunk_doc)
        # Store the similarity score in the dictionary
        similarity_scores[chunk] = similarity

    # Sort the dictionary by the similarity scores
    sorted_scores = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)

    # Return the most similar chunk
    return [sorted_scores[i][0] for i in range(k)]

In [58]:
print("\n\n".join(find_most_similar_chunk([t.page_content for t in textssel], "Que concluyó de su padre?")))

1. De mi abuelo Vero: el buen carácter y la serenidad.
2. De la reputación y memoria legadas por mi progenitor: el carácter
discreto y viril.
3. De mi madre: el respeto a los dioses, la generosidad y la abstención no
sólo de obrar mal, sino incluso de incurrir en semejante pensamiento; más
todavía, la frugalidad en el régimen de vida y el alejamiento del modo de
vivir propio de los ricos.
4. De mi bisabuelo: el no haber frecuentado las escuelas públicas y
haberme servido de buenos maestros en casa, y el haber comprendido que,
para tales fines, es preciso gastar con largueza.
5. De mi preceptor: el no haber sido de la facción de los Verdes ni de los
Azules, ni partidario de los parinularios ni de los escutarios; el soportar las
fatigas y tener pocas necesidades; el trabajo con esfuerzo personal y la
abstención de excesivas tareas, y la desfavorable acogida a la calumnia.
6. De Diogneto: el evitar inútiles ocupaciones; y la desconfianza en lo que

13. De Catulo: el no dar poca importanci